<a href="https://colab.research.google.com/github/Hanish2004/Google-Collab/blob/main/model_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import os
%matplotlib inline

In [2]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,BatchNormalization,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img,img_to_array

In [3]:
! git clone https://github.com/laxmimerit/male-female-face-dataset.git

Cloning into 'male-female-face-dataset'...
remote: Enumerating objects: 58657, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 58657 (delta 0), reused 0 (delta 0), pack-reused 58656
Receiving objects: 100% (58657/58657), 261.44 MiB | 28.00 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Updating files: 100% (58659/58659), done.


In [5]:
epochs=50
lr=1e-3
batch_size=128
data=[]
labels=[]

In [7]:
size=224 #height/width

In [8]:
train_datagen=ImageDataGenerator(horizontal_flip=True,
                                 width_shift_range=0.4,
                                 height_shift_range=0.4,
                                 zoom_range=0.3,
                                 rotation_range=20,
                                 rescale=1/255)

In [9]:
test_gen=ImageDataGenerator(rescale=1/255)

In [10]:
target_size=(size,size)
target_size

(224, 224)

In [11]:
train_generator=train_datagen.flow_from_directory(
    directory='/content/male-female-face-dataset/Training',
    target_size=target_size,
    batch_size=batch_size,
    class_mode='binary'
)

Found 47009 images belonging to 2 classes.


In [25]:
validation_generator=test_gen.flow_from_directory(
    directory='/content/male-female-face-dataset/Validation',
    target_size=target_size,
    batch_size=batch_size,
    class_mode='binary'
)

Found 11649 images belonging to 2 classes.


In [13]:
train_generator.class_indices

{'female': 0, 'male': 1}

In [14]:
len(train_generator.classes)

47009

In [15]:
train_generator.class_mode

'binary'

In [16]:
x,y=train_generator.next()

In [17]:
x[0].shape

(224, 224, 3)

In [18]:
x[0]

array([[[0.30054137, 0.15722367, 0.0973174 ],
        [0.29411766, 0.15294118, 0.10588236],
        [0.29411766, 0.15294118, 0.10588236],
        ...,
        [0.2963289 , 0.14755361, 0.07396311],
        [0.37273476, 0.21304435, 0.14218263],
        [0.3803922 , 0.21960786, 0.14901961]],

       [[0.29674098, 0.15469006, 0.10238461],
        [0.29411766, 0.15294118, 0.10588236],
        [0.2749325 , 0.13309449, 0.0886819 ],
        ...,
        [0.3317994 , 0.1779569 , 0.10563321],
        [0.3803922 , 0.21960786, 0.14901961],
        [0.3803922 , 0.21960786, 0.14901961]],

       [[0.29411766, 0.15294118, 0.10588236],
        [0.29411766, 0.15294118, 0.10588236],
        [0.23819521, 0.09509038, 0.055745  ],
        ...,
        [0.3672699 , 0.20836018, 0.13730331],
        [0.3803922 , 0.21960786, 0.14901961],
        [0.3803922 , 0.21960786, 0.14901961]],

       ...,

       [[0.8871905 , 0.66353023, 0.59673274],
        [0.92655045, 0.6903023 , 0.62363565],
        [0.9607844 , 0

In [19]:
model=Sequential()
model.add(InceptionV3(include_top=False,pooling='avg',weights='imagenet'))
model.add(Flatten())

model.add(BatchNormalization())
model.add(Dense(2048,activation='relu'))
model.add(BatchNormalization())

model.add(Dense(1024,activation='relu'))
model.add(BatchNormalization())

model.add(Dense(1,activation='sigmoid'))

model.layers[0].trainable=False

87910968/87910968 [==============================] - 1s 0us/step


In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 2048)              21802784  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 batch_normalization_94 (Ba  (None, 2048)              8192      
 tchNormalization)                                               
                                                                 
 dense (Dense)               (None, 2048)              4196352   
                                                                 
 batch_normalization_95 (Ba  (None, 2048)              8192      
 tchNormalization)                                               
                                                                 
 dense_1 (Dense)             (None, 1024)              2

In [22]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [23]:
len((train_generator.filenames)),batch_size,len((train_generator.filenames))//batch_size,epochs


(47009, 128, 367, 50)

In [ ]:
model.fit(train_generator,steps_per_epoch=len(train_generator.filenames)//batch_size,
          epochs=2,validation_data=validation_generator,validation_steps=len(validation_generator.filenames)//batch_size)

Epoch 1/2
  1/367 [..............................] - ETA: 2:11:31 - loss: 0.2841 - accuracy: 0.8672

KeyboardInterrupt: ignored